In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup

import time

In [2]:
def get_content_from_new_tab(driver,script: str,element_id: str):
    driver.execute_script(script)
    alert_obj = driver.switch_to.alert
    alert_obj.accept()
    driver.switch_to.window(driver.window_handles[-1])
    
    element = None
    while not element:
        try:
            element = driver.find_element_by_id(element_id)
        except:
            print('.',end='')
            time.sleep(1)

    table_content = element.get_attribute('innerHTML')
    driver.close()
    return table_content

def get_data(table_content,tag: str):
    soup = BeautifulSoup(table_content)    
    data_cols = []
    for table_data in soup.find_all(tag):
        data = table_data.text.strip(':').strip()
        if data:
            data_cols.append(data)
    return data_cols

In [3]:
tipo_consulta = 'Repasse'
ano = '2019'
UF = 'BA'
UF_nome = 'Bahia'
programa = 'ALIMENTAÇÃO ESCOLAR'

form_input = {
    'cbTipoConsulta': tipo_consulta, # Tipo_de_OPC
    'cbAno': ano,                    # Ano Inicio Vigencia
    'cbPrograma': programa,          # Programa/Projeto
    'cbUf': UF_nome                  # UF
}

fname = 'PNAE'
for value in form_input.values():
    fname += '_' + value.replace(' ','_')
fname += '.csv'
print('Arquivo de saída:',fname)

Arquivo de saída: PNAE_Repasse_2019_ALIMENTAÇÃO_ESCOLAR_Bahia.csv


In [4]:
driver = webdriver.Firefox()

url = 'https://www.fnde.gov.br/sigpcadm/sistema.pu?operation=localizar'

query_str = f'#consulta=tipoConsulta%3D2%26anoConcessao%3D{ano}%26programaId%3DC7%26ufId%3D{UF}'

driver.get(url+query_str)

In [5]:
## get current_page
element_id = 'ext-comp-1004' # número da página atual
curren_page_elm = None
while not curren_page_elm:
    try:
        curren_page_elm = driver.find_element_by_id(element_id)
    except:
        print('.',end='')
        time.sleep(1)

current_page = ''
while not current_page:
    current_page = curren_page_elm.get_attribute('value')
    print('.',end='')
    time.sleep(1)
current_page = int(current_page)
new_page = current_page

## get total_pages
element_id = 'ext-comp-1005' # total de páginas
element = driver.find_element_by_id(element_id)
total_pages = int(element.text.split()[-1])

scripts = []
while current_page <= total_pages:
    print(f'\n Analisando página {current_page} de {total_pages}',end='  ')
    element_id = 'ext-gen240' # tabela com resultados (body)
    element = driver.find_element_by_id(element_id)
    table_content = element.get_attribute('innerHTML')
    soup = BeautifulSoup(table_content)
    for a in soup.find_all('a'):
        href = a['href']
        if 'abrirNoPconline' in href:
            scripts.append(href)
    
    if current_page == total_pages:
        print(f'\n Foram lidos {len(scripts)} scripts.')
        break
    ## click to next page
    element_id = 'ext-gen265'  # botão "next page"
    element = driver.find_element_by_id(element_id)
    element.click()
    
    ## update current_page
    while new_page == current_page:
        time.sleep(1)
        print('.',end='')
        new_page = int(curren_page_elm.get_attribute('value'))
    current_page = new_page

............
 Analisando página 1 de 11  ..................
 Analisando página 2 de 11  ...........
 Analisando página 3 de 11  ...........
 Analisando página 4 de 11  ...............
 Analisando página 5 de 11  .................
 Analisando página 6 de 11  ..........
 Analisando página 7 de 11  ..........
 Analisando página 8 de 11  .........
 Analisando página 9 de 11  ..........
 Analisando página 10 de 11  .........
 Analisando página 11 de 11  
 Foram lidos 418 scripts.


In [6]:
sep = ','

## get headers
driver.switch_to.window(driver.window_handles[0])

script = scripts[0]
element_id='div_direita'
content = get_content_from_new_tab(driver,script,element_id)
cols = get_data(content,tag='th')
csv_content = sep.join(cols) + '\r\n'

..

In [7]:
## get data

for i,script in enumerate(scripts):
    print('\n iteração:',i,end='  ')
    driver.switch_to.window(driver.window_handles[0])
    element_id = 'div_direita'
    content = get_content_from_new_tab(driver,script,element_id)
    cols = get_data(content,tag='td')
    csv_content += sep.join(cols) + '\r\n'


 iteração: 0  ....
 iteração: 1  ..
 iteração: 2  .
 iteração: 3  .
 iteração: 4  .
 iteração: 5  .
 iteração: 6  .
 iteração: 7  .
 iteração: 8  .
 iteração: 9  ..
 iteração: 10  .
 iteração: 11  .
 iteração: 12  .
 iteração: 13  .
 iteração: 14  ..
 iteração: 15  ..
 iteração: 16  .
 iteração: 17  ..
 iteração: 18  .
 iteração: 19  .
 iteração: 20  .
 iteração: 21  .
 iteração: 22  ..
 iteração: 23  .
 iteração: 24  .
 iteração: 25  .
 iteração: 26  .
 iteração: 27  ..
 iteração: 28  .
 iteração: 29  .
 iteração: 30  .
 iteração: 31  .
 iteração: 32  .
 iteração: 33  .
 iteração: 34  .
 iteração: 35  ..
 iteração: 36  ..
 iteração: 37  ..
 iteração: 38  .
 iteração: 39  .
 iteração: 40  .
 iteração: 41  .
 iteração: 42  .
 iteração: 43  .
 iteração: 44  ...
 iteração: 45  ..
 iteração: 46  .
 iteração: 47  .
 iteração: 48  .
 iteração: 49  .
 iteração: 50  .
 iteração: 51  ..
 iteração: 52  .
 iteração: 53  ..
 iteração: 54  .
 iteração: 55  ..
 iteração: 56  .
 iteração: 57  .
 ite

In [8]:
if scripts:
    scripts_file = 'scripts_' + fname.split('.')[0] + '.txt'
    f = open(scripts_file, "w")
    f.write('\r\n'.join(scripts))
    f.close()

f = open(fname, "w")
f.write(csv_content)
f.close()


In [9]:
# scripts_file = 'scripts_' + fname.split('.')[0] + '.txt'
# f = open(scripts_file, "r")
# lines = f.readlines()
# f.close()
# '\r\n'.join([line.strip() for line in lines])

## Sites pesquisados
- https://towardsdatascience.com/web-scraping-basics-82f8b5acd45c
- https://selenium-python.readthedocs.io/
- https://www.selenium.dev/documentation/en/webdriver/waits/
- https://pythonbasics.org/selenium-execute-javascript/
- https://www.techbeamers.com/handle-alert-popup-selenium-python/
- https://stackoverflow.com/questions/28715942/how-do-i-switch-to-the-active-tab-in-selenium
- https://stackoverflow.com/questions/51443387/selenium-python-using-actionchains
- https://stackoverflow.com/questions/35385221/send-multiple-tab-key-presses-with-selenium
- 



